Broadcast переменные используются для эффективного распределения больших объектов или небольших наборов данных по всем узлам кластера. Это позволяет избежать многократной передачи одних и тех же данных при выполнении распределенных вычислений. Broadcast переменные полезны, когда у вас есть неизменяемые данные, которые нужно использовать в нескольких местах в вашем приложении Spark, и эти данные достаточно велики, чтобы их передача в каждый узел была неэффективной.

Ничего не понятно, но очень интересно... Давайте разбираться.

Я предвкушаю Ваши вопросы по поводу того, а в чем разница в сравнении с cache и с persist соответственно? В ходе объяснения постараюсь рассказать Вам эту разницу.

Broadcast чаще всего используется в случае, когда у Вас есть какой-то справочник с данными. То есть он не будет уже изменяться. И, поскольку, он хранится на всех узлах одновременно, то у нас нет необходимости каждый раз вызывать этап shuffle для перемещения ключей. Приведу простой пример, когда это может использоваться. 

Например, если у вас есть большой DataFrame и маленький DataFrame, и вы хотите выполнить join, оптимальным решением будет использовать broadcast переменную для маленького DataFrame. Это позволяет избежать дорогих shuffle операций, так как маленький DataFrame будет распространяться по всем узлам и join будет выполняться локально на каждом узле.

In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import broadcast, col


spark = SparkSession.builder \
    .appName("Broadcast Join Example") \
    .getOrCreate()

In [3]:
# Пример большого DataFrame
large_data = [(i, f"Name_{i % 5}") for i in range(1000000)]
large_df = spark.createDataFrame(large_data, ["id", "name"])

In [6]:
large_df.show(3)

+---+------+
| id|  name|
+---+------+
|  0|Name_0|
|  1|Name_1|
|  2|Name_2|
+---+------+
only showing top 3 rows



24/09/17 20:59:46 WARN TaskSetManager: Stage 1 contains a task of very large size (3791 KiB). The maximum recommended task size is 1000 KiB.


In [7]:
# Пример маленького DataFrame
small_data = [(1, "HR"), (2, "Finance"), (3, "Engineering")]
small_df = spark.createDataFrame(small_data, ["id", "department"])
small_df.show()

+---+-----------+
| id| department|
+---+-----------+
|  1|         HR|
|  2|    Finance|
|  3|Engineering|
+---+-----------+



In [9]:
# Применение broadcast к маленькому DataFrame
broadcast_df = broadcast(small_df)

In [20]:
j_df = large_df.join(broadcast_df, large_df.id == broadcast_df.id)

In [21]:
j_df.show()

24/09/17 21:08:37 WARN TaskSetManager: Stage 5 contains a task of very large size (3791 KiB). The maximum recommended task size is 1000 KiB.
24/09/17 21:08:38 WARN TaskSetManager: Stage 6 contains a task of very large size (3919 KiB). The maximum recommended task size is 1000 KiB.


+---+------+---+-----------+
| id|  name| id| department|
+---+------+---+-----------+
|  1|Name_1|  1|         HR|
|  2|Name_2|  2|    Finance|
|  3|Name_3|  3|Engineering|
+---+------+---+-----------+

